In [ ]:
!pip install flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import tensorflow as tf
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

class QLearningModel:
    def __init__(self, n_features, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.n_features = n_features

    def get_q_value(self, state, action):
        return self.q_table.get((state, action), 0.0)

    def update_q_table(self, state, action, reward, next_state):
        best_next_action = max([self.get_q_value(next_state, a) for a in range(2)], default=0)
        current_q_value = self.get_q_value(state, action)
        new_q_value = current_q_value + self.alpha * (reward + self.gamma * best_next_action - current_q_value)
        self.q_table[(state, action)] = new_q_value

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice([0, 1])  # Randomly choose between like (1) and dislike (0)
        else:
            like_value = self.get_q_value(state, 1)
            dislike_value = self.get_q_value(state, 0)
            return 1 if like_value > dislike_value else 0

    def get_state(self, product):
        return tuple(product[f'feature{i}'] for i in range(1, self.n_features + 1))

    def recommend(self, product, products_df):
        similar_products = products_df[
            (products_df['category'] == product['category']) &
            (products_df['subcategory'] == product['subcategory']) |
            (products_df['color'] == product['color']) |
            (products_df['occasion'] == product['occasion'])
        ]
        recommendations = similar_products.sample(10)
        return recommendations.to_dict(orient='records')

n_features = 5
model = QLearningModel(n_features)

@app.route('/initialize', methods=['POST'])
def initialize():
    data = request.get_json()
    initial_dataset = data['initial_dataset']

    global products_df
    products_df = pd.DataFrame(initial_dataset)

    return jsonify({"status": "initialized"}), 200

@app.route('/feedback', methods=['POST'])
def feedback():
    data = request.get_json()
    product = data['product']
    liked = data['liked']
    reward = 1 if liked else 0

    state = model.get_state(product)
    action = 1 if liked else 0
    next_state = state

    model.update_q_table(state, action, reward, next_state)

    recommendations = model.recommend(product, products_df)
    return jsonify(recommendations)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
!pip install flask-ngrok
!pip install Flask
